In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sys

sys.path.insert(0, "../Notebooks/Functions")
from StatsFunctions import MyFunctions as MF


In [2]:
DF = pd.read_csv("../Data/avgIQpercountry.csv")

In [7]:
DF.rename({"Average IQ": "IQ", "Literacy Rate": "Literacy", "Nobel Prices": "Nobels", "HDI (2021)": "HDI", "Mean years of schooling - 2021": "SchoolYears", "GNI - 2021": "GNI", "Population - 2023": "Population"}, axis=1, inplace=True)

,Rank,Country,IQ,Continent,Literacy,Nobels,HDI,SchoolYears,GNI - 2021,Population
0,1,Japan,106.48,Asia,0.99,29,0.925,13.4,42274.0,123294513
1,2,Taiwan,106.47,Asia,0.96,4,NaN,NaN,NaN,10143543
2,3,Singapore,105.89,Asia,0.97,0,0.939,11.9,90919.0,6014723
3,4,Hong Kong,105.37,Asia,0.94,1,0.952,12.2,62607.0,7491609
4,5,China,104.10,Asia,0.96,8,0.768,7.6,17504.0,1425671352
...,...,...,...,...,...,...,...,...,...,...
188,189,Gambia,52.68,Africa,0.58,0,0.500,4.6,2172.0,2773168
189,190,Guatemala,47.72,Central America,0.79,2,0.627,5.7,8723.0,18092026
190,191,Liberia,45.07,Africa,0.48,2,0.481,5.1,1289.0,5418377
191,192,Sierra Leone,45.07,Africa,0.48,0,0.477,4.6,1622.0,8791092
